# National Flood Insurance Program Data
## Claims Sample Analysis

In [1]:
# importing libraries
import os
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['savefig.facecolor'] = 'white'
%matplotlib inline

In [2]:
print('printing packages and versions:\n')

%reload_ext watermark
%watermark -v -p numpy,pandas,geopandas,matplotlib,seaborn

printing packages and versions:

Python implementation: CPython
Python version       : 3.8.13
IPython version      : 8.4.0

numpy     : 1.23.1
pandas    : 1.4.3
geopandas : 0.11.1
matplotlib: 3.5.2
seaborn   : 0.11.2



# Preview Full Data

In [3]:
%%time

path = 'data/'
all_files = glob.glob(os.path.join(path, "claims-*.json"))

# concat all counties into one dataframe
df = pd.concat((pd.read_json(file) for file in all_files), ignore_index=True)

# reverse column order
df = df.loc[:, df.columns[::-1]]

# examine data
print('shape of data: {}'.format(df.shape))
df.head()

shape of data: (43833, 73)
CPU times: user 2.32 s, sys: 362 ms, total: 2.68 s
Wall time: 2.69 s


,id,longitude,latitude,censusBlockGroupFips,censusTract,countyCode,reportedZipCode,reportedCity,state,rentalPropertyIndicator,...,baseFloodElevation,elevationDifference,elevationCertificateIndicator,elevatedBuildingIndicator,dateOfLoss,crsClassificationCode,policyCount,basementEnclosureCrawlspaceType,asOfDate,agricultureStructureIndicator
0,7279caf2-e180-474f-9072-ce5d6382278a,-73.8,40.8,3.600501e+11,3.600501e+10,36005,10465,Currently Unavailable,NY,False,...,NaN,NaN,NaN,False,2021-09-02T00:00:00.000Z,NaN,1,0.0,2022-05-19T15:56:42.000Z,False
1,e5f8240c-06d7-48f9-b148-82040a1e5261,-73.8,40.8,3.600501e+11,3.600501e+10,36005,10465,Currently Unavailable,NY,False,...,NaN,NaN,NaN,False,2012-10-29T00:00:00.000Z,NaN,1,1.0,2020-01-22T16:55:53.000Z,False
2,abc053a4-1d37-420c-8fba-a2c97ef52dda,-73.8,40.8,3.600502e+11,3.600502e+10,36005,10465,Currently Unavailable,NY,False,...,NaN,NaN,1.0,False,2017-11-22T00:00:00.000Z,NaN,1,1.0,2020-01-22T16:55:53.000Z,False
3,7764e429-a818-48e4-ba0a-99bb50a67f5c,-73.8,40.8,3.600501e+11,3.600501e+10,36005,10465,Currently Unavailable,NY,False,...,NaN,NaN,NaN,False,2012-10-29T00:00:00.000Z,NaN,1,2.0,2020-01-22T16:55:53.000Z,False
4,a5e6a2e2-4798-47b1-bfcd-cdae0cc1d11b,-73.8,40.9,3.600505e+11,3.600505e+10,36005,10464,Currently Unavailable,NY,False,...,NaN,NaN,NaN,False,2012-10-29T00:00:00.000Z,NaN,1,1.0,2020-01-22T16:55:53.000Z,False


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43833 entries, 0 to 43832
Data columns (total 73 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   id                                          43833 non-null  object 
 1   longitude                                   43833 non-null  float64
 2   latitude                                    43833 non-null  float64
 3   censusBlockGroupFips                        43416 non-null  float64
 4   censusTract                                 43416 non-null  float64
 5   countyCode                                  43833 non-null  int64  
 6   reportedZipCode                             43833 non-null  object 
 7   reportedCity                                43833 non-null  object 
 8   state                                       43833 non-null  object 
 9   rentalPropertyIndicator                     43833 non-null  bool   
 10  buildingDe

# Data Wrangling